# 引言

分词器是每个大语言模型必不可少的组件，但每个大语言模型的分词器几乎都不相同。如果要训练自己的分词器，可以使用huggingface的tokenizers框架，tokenizers包含以下主要组件：

- Tokenizer: 分词器的核心组件，定义了分词的整个流程，包括标准化、预分词、模型分词、后处理等
- Normalizers：可选，负责将文本标准化，包括unicode归一化、大写转小写、去重音等操作
- Pre-tokenizers：负责将文本分割成更小的片段（如单词等），为模型分词做准备。常见的预分词器有按空格分词（Whitespace）、正则表达式分词（Regex）等
- Models：是实际的分词算法，负责将文本片段转换为子词，常见的有BPE、WordPiece、Unigram等。
- Post-Processors：负责对分词结果进行后处理，如添加特殊标记（CLS、SEP）。
- Decoders：负责将分词结果转换回原始文本，常见的解码器有 ByteLevel、WordPiece 等。
- Trainers：用于训练分词模型，不同的模型对应不同的训练器，如 BpeTrainer、WordPieceTrainer、UnigramTrainer 等。

在开始之前，先导入对应的包。

In [1]:
import json
import re
import os
from tokenizers import (
    Tokenizer,
    normalizers,
    pre_tokenizers,
    models,
    processors,
    decoders,
    trainers
)

# 加载语料库

这里的语料库是jsonl文件，每一行是一条json个格式的文本数据

定义一个函数用于从JSONL文件中读取文本数据，考虑到语料库会比较大，所以采用yield生成器来延迟到访问时再加载数据。

In [2]:
def read_texts_from_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as fr:
        for i, line in enumerate(fr):
#             if i == 0: print(line)
            if i >= 100000: break
            data = json.loads(line)
            yield data['text']

In [3]:
data_path = "./datasets/pretrain_hq.jsonl"
texts = read_texts_from_jsonl(data_path)
type(texts)

generator

可以看到，函数read_texts_from_jsonl返回的并不是真正的数据，而是一个生成器generator。可以通过next函数像访问iterator一样访问迭代数据。

In [4]:
next(texts)

'<s>鉴别一组中文文章的风格和特点，例如官方、口语、文言等。需要提供样例文章才能准确鉴别不同的风格和特点。</s> <s>好的，现在帮我查一下今天的天气怎么样?今天的天气依据地区而异。请问你需要我帮你查询哪个地区的天气呢？</s> <s>打开闹钟功能，定一个明天早上七点的闹钟。好的，我已经帮您打开闹钟功能，闹钟将在明天早上七点准时响起。</s> <s>为以下场景写一句话描述：一个孤独的老人坐在公园长椅上看着远处。一位孤独的老人坐在公园长椅上凝视远方。</s> <s>非常感谢你的回答。请告诉我，这些数据是关于什么主题的？这些数据是关于不同年龄段的男女人口比例分布的。</s> <s>帮我想一个有趣的标题。这个挺有趣的："如何成为一名成功的魔术师" 调皮的标题往往会吸引读者的注意力。</s> <s>回答一个问题，地球的半径是多少？地球的平均半径约为6371公里，这是地球自赤道到两极的距离的平均值。</s> <s>识别文本中的语气，并将其分类为喜悦、悲伤、惊异等。\n文本：“今天是我的生日！”这个文本的语气是喜悦。</s>'

# 训练过程
## 模型选择
使用BPE模型来初始化Tokenizer实例

In [5]:
tokenizer = Tokenizer(models.BPE())

BPE是一种基于子词的分词方法，例如：

- cats -> cat + s
- helpful -> help + ful
- congratulation -> con + gr + at + ulation

这种基于子词的分词方法，相比基于完整单词和基于单个字符有以下好处：

1. 子词相比于单词（可以认为多个子词的组合）数量要可控，这能避免词表过大，并且能避免生僻词带来的未知令牌问题。
2. 子词相比于字符语义性更强，像单个字符f是没有语义的，但子词ful可以表达满的，比较像英语里的词根词缀。

## 预分词器选择
为tokenizer设置预分词器，预分词器有以下几种：
- Whitespace：按空格分隔，粒度为单词，适用于空格分隔的语言，例如英语。
- Regex：按自定义正则表达式分隔，适用于需要自定义复杂分词规则的场景。
- ByteLevel：按字节分割，适用于特殊字符、非英语场景（例如中文）。

由于我们主要面向中文，所以这里采用ByteLevel的pre_tokenizer。

In [6]:
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
tokenizer.pre_tokenizer

pre_tokenizer对文本做了哪些处理，通过以下几个例子查看

1. 处理英文文本

In [7]:
tokenizer.pre_tokenizer.pre_tokenize_str("Pre-tokenize a :class:`~tokenizers.PyPreTokenizedString` in-place")

[('Pre', (0, 3)),
 ('-', (3, 4)),
 ('tokenize', (4, 12)),
 ('Ġa', (12, 14)),
 ('Ġ:', (14, 16)),
 ('class', (16, 21)),
 (':`~', (21, 24)),
 ('tokenizers', (24, 34)),
 ('.', (34, 35)),
 ('PyPreTokenizedString', (35, 55)),
 ('`', (55, 56)),
 ('Ġin', (56, 59)),
 ('-', (59, 60)),
 ('place', (60, 65))]

可以看到，pre_tokenizer将文本按照空格和特殊字符作了初步分词，空格处理成了特殊字符Ġ，并记录了每个词的起始和结束位置。

2. 处理中文文本

In [8]:
zh_sentence = "在查处虚开增值税专用发票案件中，常常涉及进项留抵税额和税款损失的认定和处理。"
tokenizer.pre_tokenizer.pre_tokenize_str(zh_sentence)

[('åľ¨æŁ¥å¤ĦèĻļå¼Ģå¢ŀåĢ¼ç¨İä¸ĵçĶ¨åıĳç¥¨æ¡Īä»¶ä¸Ń', (0, 15)),
 ('ï¼Į', (15, 16)),
 ('å¸¸å¸¸æ¶īåıĬè¿Ľé¡¹çķĻæĬµç¨İé¢ĿåĴĮç¨İæ¬¾æįŁå¤±çļĦè®¤å®ļåĴĮå¤ĦçĲĨ', (16, 37)),
 ('ãĢĤ', (37, 38))]

中文基本也是按照特殊符号，和。进行了分词，但分词的结果是一堆不认识的字符，这些字符是如何产生的呢？

## 构建训练器
BPE训练器中需要指定几个参数：
- vocab_size: 训练后词表中的词条数据，BPE是一个从短词到长词的组合过程，达到词表大小后就会停止训练。
- special_tokens:特殊token，和语言模型的特殊token一样。例如开始、结束、填充
- initial_alphabet:初始化字符表，使用上面长度为256的unicode字节编码表作为初始字符表。

通过pre_tokenizers.ByteLevel.alphabet()可以获得初始字符编码表。

In [9]:
json.dumps(pre_tokenizers.ByteLevel.alphabet(), ensure_ascii=False)

'["!", ")", "Ü", "ı", "ĺ", "ú", "Ķ", "y", "M", "(", "ä", "¿", "à", "®", "ô", "ì", "ÿ", "0", "í", "ñ", "ĩ", "Ä", "ç", "Ĳ", "Ę", "´", "º", "¨", "]", "Ď", "Â", "ý", "±", "¯", "_", "Ĕ", "Ń", "9", "\\"", "¬", "¶", "ĝ", "Ğ", "Ì", "Ĵ", "\\\\", "Č", "t", "`", "l", "đ", "m", "\'", "Ł", "v", "ĥ", "¹", "2", "¼", "©", "ê", "ī", "c", "3", "Ú", "Ą", "ġ", "g", "ã", "ĭ", "è", "Ĝ", "$", "ĕ", "Ľ", "Õ", "Á", "Ĭ", "»", "Å", "ð", ".", "Y", "P", "Ā", "8", "h", "µ", "C", "Ć", "ē", "ł", "5", "F", "w", "<", "I", "Ġ", "i", "Þ", "Ù", "1", "³", "Ĉ", "Ô", "B", "s", "x", "ĉ", "þ", "R", "û", "S", "Ċ", "&", "T", "ć", "|", "ą", "õ", "Ò", "Ē", "{", "Ç", ",", "?", "V", "¦", "¾", "*", "×", "ŀ", "î", "ě", "W", "ė", "Ã", "É", "ö", "Ð", "E", "d", "ù", "§", "â", "÷", "Î", "H", "Ê", "j", "#", "e", "u", "Z", "Ñ", "é", "}", "Ļ", "Į", "İ", "U", "Ħ", "ľ", "G", "¤", "·", "K", "ò", "ï", ":", "N", "å", "Ï", "/", "č", "D", "ă", "ĳ", "¥", "¸", "+", "ª", "ß", "~", "n", "Ĺ", "ģ", "ĵ", "a", "ó", "ë", "«", "ċ", "%", "@", "į", "4", "æ", "r

定义特殊token，分别为填充、开始、结束标记

In [10]:
speicial_tokens = ["<|endoftext|>", "<|im_start|>", "<|im_end|>"]

构建训练器，词条数量设置为32000

In [11]:
trainer = trainers.BpeTrainer(
    vocab_size=32000,
    special_tokens=speicial_tokens,
    show_progress=True,
    initial_alphabet=pre_tokenizers.ByteLevel.alphabet()
)

In [12]:
tokenizer.train_from_iterator(texts, trainer=trainer)

# 保存训练结果

在保存结果之前，需要先设置相匹配的解码器，否则ASCII以外的字符可能无法正常解码。

> 上面编码阶段使用了ByteLevel的预分词器，相对应的解码阶段也需要使用ByteLevel，表示将token id转换为token后，还需要进行一次unicode字节级别的解码，才能正常显示中文等多语言字符。

In [13]:
tokenizer.decoder = decoders.ByteLevel()

In [14]:
# 将训练的分词器保存到指定目录。

tokenizer_dir = "./tokenizer"
os.makedirs(tokenizer_dir, exist_ok=True)
tokenizer.save(os.path.join(tokenizer_dir, "tokenizer.json"))
tokenizer.model.save(tokenizer_dir)

['./tokenizer\\vocab.json', './tokenizer\\merges.txt']

还需要一个分词器配置文件，包括模型类型、是否使用小写字母等。

In [15]:
config = {
    "add_bos_token": False,
    "add_eos_token": False,
    "add_prefix_space": True,
    "added_tokens_decoder": {
        "0": {
            "content": "<|endoftext|>",
            "lstrip": False,
            "normalized": False,
            "rstrip": False,
            "single_word": False,
            "special": True
        },
        "1": {
            "content": "<|im_start|>",
            "lstrip": False,
            "normalized": False,
            "rstrip": False,
            "single_word": False,
            "special": True
        },
        "2": {
            "content": "<|im_end|>",
            "lstrip": False,
            "normalized": False,
            "rstrip": False,
            "single_word": False,
            "special": True
        }
    },
    "additional_special_tokens": [],
    "bos_token": "<|im_start|>",
    "clean_up_tokenization_spaces": False,
    "eos_token": "<|im_end|>",
    "legacy": True,
    "model_max_length": 1000000000000000019884624838656,
    "pad_token": None,
    "sp_model_kwargs": {},
    "spaces_between_special_tokens": False,
    "tokenizer_class": "PreTrainedTokenizerFast",
    "unk_token": "<|endoftext|>",
    "use_default_system_prompt": False,
    "chat_template": "{% if messages[0]['role'] == 'system' %}{% set system_message = messages[0]['content'] %}{% endif %}{% if system_message is defined %}{{ system_message }}{% endif %}{% for message in messages %}{% set content = message['content'] %}{% if message['role'] == 'user' %}{{ '<|im_start|>user\\n' + content + '<|im_end|>\\n<|im_start|>assistant\\n' }}{% elif message['role'] == 'assistant' %}{{ content + '<|im_end|>' + '\\n' }}{% endif %}{% endfor %}"
}

In [16]:
with open(os.path.join(tokenizer_dir, "tokenizer_config.json"), "w", encoding="utf-8") as config_file:
    json.dump(config, config_file, ensure_ascii=False, indent=4)

print("Tokenizer training completed and saved.")

Tokenizer training completed and saved.


1. vocab.json：词汇表文件，包含词条和对应的索引。
2. merges.txt: 合并表文件，定义了子词的合并规则。
3. tokenizer.json: 完整的分词器文件，它包含了分词器的所有信息，包括词汇表、合并规则、特殊标记等。
4. tokenizer_config.json: 分词器配置文件，包括了起始token、结束token的定义，以及提示词模板。

# 测试分词器

In [17]:
from transformers import AutoTokenizer

# 加载预训练的tokenizer
tokenizer_dir = "./tokenizer"
tokenizer_trained = AutoTokenizer.from_pretrained(tokenizer_dir)

1. 英文分词

In [19]:
text_en = "Pre-tokenize a :class:`~tokenizers.PyPreTokenizedString` in-place"
tokenized = tokenizer_trained.tokenize(text_en)
tokenized

['P',
 're',
 '-',
 't',
 'ok',
 'en',
 'i',
 'z',
 'e',
 'Ġ',
 'a',
 'Ġ',
 ':',
 'c',
 'l',
 'as',
 's',
 ':',
 '`',
 '~',
 't',
 'ok',
 'en',
 'i',
 'z',
 'er',
 's',
 '.',
 'P',
 'y',
 'P',
 're',
 'T',
 'ok',
 'en',
 'i',
 'z',
 'e',
 'd',
 'S',
 't',
 'r',
 'ing',
 '`',
 'Ġ',
 'in',
 '-',
 'p',
 'l',
 'a',
 'ce']

tokenize方法只对输入文本作了分词，返回的是一组明文的token。要想直接返回token_id，需要使用encode方法，分词的同时完成文本到数字的序列化。

In [20]:
token_ids_en = tokenizer_trained.encode(text_en)
token_ids_en

[50,
 27224,
 15,
 86,
 29472,
 16889,
 75,
 92,
 71,
 223,
 67,
 223,
 28,
 69,
 78,
 22605,
 85,
 28,
 66,
 96,
 86,
 29472,
 16889,
 75,
 92,
 13757,
 85,
 16,
 50,
 91,
 50,
 27224,
 54,
 29472,
 16889,
 75,
 92,
 71,
 70,
 53,
 86,
 84,
 31833,
 66,
 223,
 12359,
 15,
 82,
 78,
 67,
 13717]

In [21]:
tokenizer_trained.decode(token_ids_en)

'Pre-tokenize a :class:`~tokenizers.PyPreTokenizedString` in-place'

2. 测试中文

In [24]:
text_zh = "在查处虚开增值税专用发票案件中，常常涉及进项留抵税额和税款损失的认定和处理。"
token_ids_zh = tokenizer_trained.encode(text_zh)
token_ids_zh

[304,
 906,
 670,
 2315,
 637,
 1230,
 1444,
 4134,
 1665,
 384,
 491,
 1772,
 6552,
 305,
 263,
 3740,
 3160,
 472,
 848,
 1577,
 4823,
 4134,
 2636,
 301,
 4134,
 1592,
 6922,
 265,
 1096,
 382,
 14332,
 262]

In [25]:
tokenizer_trained.decode(token_ids_zh)

'在查处虚开增值税专用发票案件中，常常涉及进项留抵税额和税款损失的认定和处理。'